In [154]:
#https://www.kaggle.com/code/ashydv/sales-prediction-simple-linear-regression
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
# Import the numpy and pandas package
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
#import pandas_datareader.data as web
import seaborn as sns
import pandas as pd
import os.path
import numpy as np
from scipy.stats import spearmanr
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (mean_squared_error,
                             mean_absolute_error,
                             mean_squared_log_error, 
                             median_absolute_error, 
                             explained_variance_score, 
                             r2_score)

In [155]:
start_time = time.time()

data = pd.DataFrame(pd.read_csv("housing.csv"))
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
0,-122.23,37.88,41,880,129.0,322,126,8.3252,NEAR BAY,452600
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,NEAR BAY,358500
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,NEAR BAY,352100
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,NEAR BAY,341300
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,NEAR BAY,342200


In [156]:
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [157]:
print(data.isnull().sum())

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
ocean_proximity         0
median_house_value      0
dtype: int64


In [158]:
data['total_bedrooms'].fillna(data['total_bedrooms'].median(), inplace=True)
print(data.isnull().sum())

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
ocean_proximity       0
median_house_value    0
dtype: int64


In [159]:
label_encoder = LabelEncoder()
data['ocean_proximity'] = label_encoder.fit_transform(data['ocean_proximity'])

names = ['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']
scaler = StandardScaler()
data[names] = scaler.fit_transform(data[names])

print(data.head())

   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0  -1.327835  1.052548            0.982143    -0.804819       -0.972476   
1  -1.322844  1.043185           -0.607019     2.045890        1.357143   
2  -1.332827  1.038503            1.856182    -0.535746       -0.827024   
3  -1.337818  1.038503            1.856182    -0.624215       -0.719723   
4  -1.337818  1.038503            1.856182    -0.462404       -0.612423   

   population  households  median_income  ocean_proximity  median_house_value  
0   -0.974429   -0.977033       2.344766                3            2.129631  
1    0.861439    1.669961       2.332238                3            1.314156  
2   -0.820777   -0.843637       1.782699                3            1.258693  
3   -0.766028   -0.733781       0.932968                3            1.165100  
4   -0.759847   -0.629157      -0.012881                3            1.172900  


In [160]:
X = data.drop("median_house_value", axis=1)
# Our descriptive feature
print("X.describe()")
print(X.describe())

y = data['median_house_value']
print(y.describe())

X.describe()
          longitude      latitude  housing_median_age   total_rooms  \
count  2.064000e+04  2.064000e+04        2.064000e+04  2.064000e+04   
mean  -6.527810e-15  1.256263e-15        8.557001e-16  1.475181e-16   
std    1.000024e+00  1.000024e+00        1.000024e+00  1.000024e+00   
min   -2.385992e+00 -1.447568e+00       -2.196180e+00 -1.207283e+00   
25%   -1.113209e+00 -7.967887e-01       -8.453931e-01 -5.445698e-01   
50%    5.389137e-01 -6.422871e-01        2.864572e-02 -2.332104e-01   
75%    7.784964e-01  9.729566e-01        6.643103e-01  2.348028e-01   
max    2.625280e+00  2.958068e+00        1.856182e+00  1.681558e+01   

       total_bedrooms    population    households  median_income  \
count    2.064000e+04  2.064000e+04  2.064000e+04   2.064000e+04   
mean    -2.823081e-16 -6.465442e-17  2.139358e-16   3.734255e-16   
std      1.000024e+00  1.000024e+00  1.000024e+00   1.000024e+00   
min     -1.277688e+00 -1.256123e+00 -1.303984e+00  -1.774299e+00   
25%    

In [161]:
print("\nLet's build our Linear regression Model\n")

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

print("\n(X_train.head())\n")
print("======================")
print(X_train.head())

regression = linear_model.LinearRegression()
regression.fit(X_train, Y_train)


Let's build our Linear regression Model


(X_train.head())

       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
15961  -1.427661  0.972957            1.856182    -0.561874       -0.598116   
1771   -1.387731  1.085321            1.061601    -0.527495       -0.588578   
16414  -0.833696  1.061912           -1.004309    -1.185280       -1.256228   
5056    0.608792 -0.754652            0.425936     1.183664        2.477841   
8589    0.588827 -0.815516            0.743768    -0.359725       -0.488431   

       population  households  median_income  ocean_proximity  
15961   -0.482568   -0.568998      -0.357869                3  
1771    -0.401327   -0.514070      -0.137527                3  
16414   -1.241111   -1.290906      -0.655694                1  
5056     1.865470    2.279397      -1.240342                0  
8589    -0.596482   -0.485299       1.823862                0  


LinearRegression()

In [162]:
print("Model Evaluation")
print("R-squared: %.3f" % regression.score(X_test, Y_test))
Y_pred = regression.predict(X_test)
print("Mean squared error: %.3f" % mean_squared_error(Y_test, Y_pred))  # Test MSE
print("Mean absolute error: %.3f" % mean_absolute_error(Y_test, Y_pred))

end_time = time.time()
time_taken = end_time - start_time
time_taken_ms = (time_taken) * 1000
print(f"Time taken for the pipeline execution: {time_taken_ms:.2f} milliseconds, {time_taken:.4f} seconds")

Model Evaluation
R-squared: 0.628
Mean squared error: 0.367
Mean absolute error: 0.441
Time taken for the pipeline execution: 284.39 milliseconds, 0.2844 seconds


In [164]:
regression.predict(np.array([[-1.327835,1.052548,0.982143,-0.804819,-0.972476,-0.974429,-0.977033,2.344766,3]]))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([1.78584383])